# Harmonizing Multi-Source Healthcare Data

Time estimate: **20** minutes

## Objectives
After completing this lab, you will be able to:
- Align healthcare datasets from multiple source systems.  
- Clean and standardize patient and encounter identifiers.  
- Map inconsistent terminologies across datasets.  
- Merge EHR, claims, and lab data into a unified view.  
- Resolve unit mismatches during data integration.


## What you will do in this lab
In this lab, you'll clean, standardize, and merge synthetic data from EHRs, insurance claims, and laboratory systems into a harmonized dataset.

You will:

- Load synthetic EHR, claims, and lab datasets.  
- Inspect and clean identifiers used across systems.  
- Standardize codes and terminology differences.  
- Resolve unit inconsistencies in lab measurements.  
- Merge datasets into a harmonized healthcare table.


## Overview
Healthcare data rarely comes from a single system. You will often need to combine information from
EHR systems, insurance claims platforms, and laboratory systems. Each source may use different
identifiers, coding standards, and measurement units. This lab focuses on the practical challenges
of harmonizing these datasets so they can be analyzed together reliably.


## About the dataset/environment
You will work with **three synthetic datasets** representing common healthcare data sources:
- EHR data with patient demographics and encounters  
- Claims data with billing and diagnosis codes  
- Lab data with test results and measurement units  

The datasets intentionally include mismatched identifiers, inconsistent codes, and unit differences.


## Setup

In [ ]:
# This cell loads multi-source healthcare datasets from CSV files.
# It ensures the lab is file-driven, reproducible, and realistic.

import pandas as pd

# -----------------
# Load EHR dataset
# -----------------
ehr_df = pd.read_csv("https://fundamentals-of-healthcare-data-science-858397.gitlab.io/labs/lab3/ehr_data.csv")

# --------------------
# Load Claims dataset
# --------------------
claims_df = pd.read_csv("https://fundamentals-of-healthcare-data-science-858397.gitlab.io/labs/lab3/claims_data.csv")

# -----------------
# Load Labs dataset
# -----------------
labs_df = pd.read_csv("https://fundamentals-of-healthcare-data-science-858397.gitlab.io/labs/lab3/labs_data.csv")




In [ ]:
# Display ehr dataset
ehr_df.head()

In [ ]:
# Display claims dataset
claims_df.head()

In [ ]:
# Display lab dataset
labs_df.head()

## Step 1: Inspect identifier differences across sources
In this step, you will compare how patients are identified across EHR, claims, and lab systems.
This helps understand why records do not automatically align.

**Why this matters in healthcare:** Inconsistent identifiers are one of the most common causes of failed data integration.


In [ ]:

# This cell displays patient identifiers from each dataset.
# Reviewing identifiers helps identify formatting differences early.

print(ehr_df["ehr_patient_id"], claims_df["claims_member_id"], labs_df["lab_patient_id"])


## Step 2: Clean and standardize patient identifiers
Here, you will clean and standardize patient identifiers so they follow a common format.
This is a prerequisite for merging datasets reliably.

**Why this matters in healthcare:** Poor identifier hygiene can result in missing or incorrect patient matches.


In [ ]:
# This cell standardizes patient identifiers across datasets.
# Consistent identifiers are required for accurate joins.

# Standardize EHR patient IDs
ehr_df["patient_id"] = ehr_df["ehr_patient_id"].str.replace("P", "")

# Standardize Lab patient IDs
labs_df["patient_id"] = labs_df["lab_patient_id"].str.replace("P-", "")

# Standardize Claims patient IDs by converting to string and padding with leading zeros
claims_df["patient_id"] = claims_df["claims_member_id"].astype(str).str.zfill(3)

ehr_df, claims_df, labs_df

## Step 3: Map inconsistent terminologies
Different systems might use different codes or labels for the same concept.
In this step, you will map diagnosis codes to human-readable descriptions.

**Why this matters in healthcare:** Unmapped codes make cross-system analysis difficult and error-prone.


In [ ]:

# This cell maps diagnosis codes to readable descriptions.
# Terminology mapping improves interpretability.

diagnosis_map = {
    "I10": "Hypertension",
    "E11": "Type 2 Diabetes"
}

claims_df["diagnosis_desc"] = claims_df["diagnosis_code"].map(diagnosis_map)

claims_df


## Step 4: Resolve unit mismatches in lab data
Lab results might be reported in different units across systems.
Here, you will convert all glucose values to a single standard unit.

**Why this matters in healthcare:** Unit mismatches can lead to clinically dangerous misinterpretations.


In [ ]:

# This cell standardizes lab result units.
# Converting to a common unit enables safe comparison.

def convert_glucose_to_mgdl(value, unit):
    # Convert glucose values to mg/dL
    if unit == "mmol/L":
        return value * 18
    return value

labs_df["glucose_mgdl"] = labs_df.apply(
    lambda row: convert_glucose_to_mgdl(row["result_value"], row["unit"]),
    axis=1
)

labs_df


## Step 5: Merge EHR, claims, and lab data
With identifiers cleaned and values standardized, you will now merge the datasets.
This produces a unified patient-level view.

**Why this matters in healthcare:** Integrated data enables richer analysis across clinical and financial domains.


In [ ]:

# This cell merges all datasets into a single table.
# Harmonized data supports comprehensive analytics.

merged_df = ehr_df.merge(claims_df, on="patient_id", how="inner")                   .merge(labs_df, on="patient_id", how="inner")

merged_df


## Step 6: Review harmonized dataset
Finally, you will review the merged dataset to confirm alignment and completeness.
This step validates the success of the harmonization process.

**Why this matters in healthcare:** Final validation prevents silent integration errors from propagating.


In [ ]:

# This cell reviews the structure and contents of the harmonized dataset.
# Validation ensures integration steps worked as expected.

merged_df.info()
merged_df


## Exercises

In [ ]:
# -----------------
# Load EHR exercise dataset
# -----------------
ehr_df = pd.read_csv("https://fundamentals-of-healthcare-data-science-858397.gitlab.io/labs/lab3/ehr_data_exercise.csv")

# --------------------
# Load Claims exercise dataset
# --------------------
claims_df = pd.read_csv("https://fundamentals-of-healthcare-data-science-858397.gitlab.io/labs/lab3/claims_data_exercise.csv")

# -----------------
# Load Labs exercise dataset
# -----------------
labs_df = pd.read_csv("https://fundamentals-of-healthcare-data-science-858397.gitlab.io/labs/lab3/labs_data_exercise.csv")

### Exercise 1: Inspect identifier formats

In [ ]:
# your code goes here


<details>
<summary>Click here for a hint</summary>

Look at how patient IDs differ across datasets.

</details>

<details>
<summary>Click here for solution</summary>

```python
print(ehr_df[['ehr_patient_id']])
print(claims_df[['claims_member_id']])
print(labs_df[['lab_patient_id']])
```

</details>


### Exercise 2: Standardize patient identifiers

In [ ]:
# your code goes here


<details>
<summary>Click here for a hint</summary>

Create a common patient_id field.

</details>

<details>
<summary>Click here for solution</summary>

```python
# Standardize EHR patient IDs: Remove 'PX' prefix
ehr_df["patient_id"] = ehr_df["ehr_patient_id"].str.replace("PX", "")

# Standardize Lab patient IDs: Remove 'PX-' prefix
labs_df["patient_id"] = labs_df["lab_patient_id"].str.replace("PX-", "")

# Standardize Claims patient IDs: Adjust value by subtracting 500, convert to string, and zero-fill to 3 digits
claims_df["patient_id"] = (claims_df["claims_member_id"] - 500).astype(str).str.zfill(3)

print(ehr_df.head())
print(labs_df.head())
```

</details>


### Exercise 3: Map diagnosis codes

In [ ]:
# your code goes here


<details>
<summary>Click here for a hint</summary>

Translate codes into descriptions.

</details>

<details>
<summary>Click here for solution</summary>

```python
dia_map = {
    "I10": "Hypertension",
    "E11": "Type 2 Diabetes",
    "E78": "Hyperlipidemia" # Added for exercise data
}

claims_df["diagnosis_desc"] = claims_df["diagnosis_code"].map(dia_map)
claims_df[['diagnosis_code','diagnosis_desc']]
```

</details>


### Exercise 4: Convert lab units

In [ ]:
# your code goes here

<details>
<summary>Click here for a hint</summary>

Convert all glucose values to mg/dL.

</details>

<details>
<summary>Click here for solution</summary>

```python
def convert_glucose_to_mgdl(value, unit):
    # Convert glucose values to mg/dL
    if unit == "mmol/L":
        return value * 18
    return value

labs_df["glucose_mgdl"] = labs_df.apply(
    lambda row: convert_glucose_to_mgdl(row["result_value"], row["unit"]),
    axis=1
)

labs_df[['result_value','unit','glucose_mgdl']]
```

</details>


### Exercise 5: Merge datasets

In [ ]:
# your code goes here

<details>
<summary>Click here for a hint</summary>

Join EHR, claims, and labs.

</details>

<details>
<summary>Click here for solution</summary>

```python
merged_df = ehr_df.merge(claims_df, on="patient_id", how="inner").merge(labs_df, on="patient_id", how="inner")

merged_df
```

</details>


### Exercise 6: Validate merged data

In [ ]:
# your code goes here

<details>
<summary>Click here for a hint</summary>

Inspect merged table structure.

</details>

<details>
<summary>Click here for solution</summary>

```python
merged_df.info()
merged_df
```

</details>


## Congratulations!

You have successfully harmonized multi-source healthcare data. You can now recognize and address common data inconsistencies across systems, an essential step before performing reliable healthcare analytics.

## Authors
Ramesh Sannareddy   

<br>   

© SkillUp. All rights reserved.
   
Materials may not be reproduced in whole or in part without written permission from SkillUp.